# French to English Translation with SentinelTranslate

This notebook demonstrates how to use SentinelTranslate to translate French text to English using the OPUS-MT (`opus-mt-fr-en`) model served via NVIDIA Triton Inference Server.

## What You'll Learn

- Connect to the SentinelTranslate API and verify services are running
- Translate single French sentences to English
- Handle common French phrases (greetings, business, technical)
- Test edge cases (numbers, dates, named entities)
- Understand hallucination detection mechanisms
- Measure translation performance and latency
- Handle errors and implement retry logic

## Prerequisites

**Services must be running**:
```bash
docker-compose up -d
```

**Required models**:
- `opus-mt-fr-en` in Triton model repository

**Estimated time**: 15-20 minutes

## Setup and Imports

In [ ]:
import requests
import json
import time
from typing import Dict, Any
from datetime import datetime

# Configuration
SIDECAR_URL = "http://localhost:8080"
BATCH_API_URL = "http://localhost:8090"
TRITON_URL = "http://localhost:8000"

# Language pair
SOURCE_LANG = "fr"  # French
TARGET_LANG = "en"  # English
MODEL_NAME = "opus-mt-fr-en"

print("✅ Imports successful")
print(f"📝 Source Language: French ({SOURCE_LANG})")
print(f"🎯 Target Language: English ({TARGET_LANG})")
print(f"🤖 Model: {MODEL_NAME}")

## Step 1: Verify Services are Running

Before we start translating, let's make sure all services are healthy.

In [ ]:
def check_service_health(service_name: str, url: str) -> bool:
    """Check if a service is healthy and responding."""
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✅ {service_name}: Healthy")
            return True
        else:
            print(f"⚠️  {service_name}: Responded with status {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ {service_name}: Not reachable - {e}")
        return False

# Check all services
print("Checking service health...\n")
sidecar_healthy = check_service_health("Sidecar API", f"{SIDECAR_URL}/health")
batch_healthy = check_service_health("Batch API", f"{BATCH_API_URL}/health")
triton_healthy = check_service_health("Triton Server", f"{TRITON_URL}/v2/health/ready")

if sidecar_healthy and triton_healthy:
    print("\n🎉 All required services are running!")
else:
    print("\n⚠️  Some services are not running. Please start them with: docker-compose up -d")

## Step 2: Helper Functions

Let's create utility functions to simplify translation requests.

In [ ]:
def translate_text(text: str, source_lang: str = SOURCE_LANG, target_lang: str = TARGET_LANG, 
                   max_wait: int = 30, poll_interval: float = 0.5) -> Dict[str, Any]:
    """
    Translate text using SentinelTranslate API.
    
    Args:
        text: Text to translate
        source_lang: Source language code (default: fr)
        target_lang: Target language code (default: en)
        max_wait: Maximum seconds to wait for result (default: 30)
        poll_interval: Seconds between status checks (default: 0.5)
    
    Returns:
        Dictionary with translation result and metadata
    """
    start_time = time.time()
    
    # Submit translation job
    payload = {
        "text": text,
        "source_lang": source_lang,
        "target_lang": target_lang
    }
    
    try:
        response = requests.post(f"{SIDECAR_URL}/translate", json=payload, timeout=10)
        response.raise_for_status()
        job_data = response.json()
        job_id = job_data["job_id"]
    except Exception as e:
        return {
            "success": False,
            "error": f"Failed to submit job: {e}",
            "source": text
        }
    
    # Poll for result
    elapsed = 0
    while elapsed < max_wait:
        try:
            status_response = requests.get(f"{SIDECAR_URL}/status/{job_id}", timeout=5)
            status_response.raise_for_status()
            result = status_response.json()
            
            if result["status"] == "SUCCESS":
                translation_time = time.time() - start_time
                return {
                    "success": True,
                    "source": text,
                    "translation": result["result"],
                    "job_id": job_id,
                    "latency_ms": round(translation_time * 1000, 2),
                    "source_lang": source_lang,
                    "target_lang": target_lang
                }
            elif result["status"] == "FAILURE":
                return {
                    "success": False,
                    "error": result.get("error", "Unknown error"),
                    "source": text,
                    "job_id": job_id
                }
            
            # Still pending or in progress
            time.sleep(poll_interval)
            elapsed = time.time() - start_time
            
        except Exception as e:
            return {
                "success": False,
                "error": f"Error checking status: {e}",
                "source": text,
                "job_id": job_id
            }
    
    return {
        "success": False,
        "error": f"Timeout after {max_wait}s",
        "source": text,
        "job_id": job_id
    }

def print_translation(result: Dict[str, Any]):
    """Pretty print translation result."""
    if result["success"]:
        print(f"🇫🇷 Source:      {result['source']}")
        print(f"🇬🇧 Translation: {result['translation']}")
        print(f"⏱️  Latency:     {result['latency_ms']} ms")
        print(f"🆔 Job ID:      {result['job_id'][:8]}...")
    else:
        print(f"❌ Error: {result['error']}")
        print(f"🇫🇷 Source: {result['source']}")

print("✅ Helper functions defined")

## Step 3: Basic Translation Examples

Let's start with a simple French sentence.

In [ ]:
# Simple greeting
print("Example 1: Simple Greeting\n" + "="*50)
result = translate_text("Bonjour, comment allez-vous?")
print_translation(result)

In [ ]:
# Longer sentence
print("\nExample 2: Longer Sentence\n" + "="*50)
result = translate_text("Je suis très heureux de vous rencontrer aujourd'hui.")
print_translation(result)

## Step 4: Common French Phrases

Let's translate common phrases you might encounter in different contexts.

In [ ]:
# Common French phrases
phrases = [
    # Greetings
    "Bonjour",
    "Bonsoir",
    "Au revoir",
    "Enchanté",
    
    # Business
    "J'aimerais planifier une réunion.",
    "Pourriez-vous m'envoyer le rapport?",
    "Merci pour votre collaboration.",
    
    # Technical
    "Le serveur ne répond pas.",
    "Veuillez redémarrer l'application.",
    "La base de données est surchargée."
]

print("Common French Phrases\n" + "="*70 + "\n")

latencies = []
for phrase in phrases:
    result = translate_text(phrase)
    if result["success"]:
        print(f"🇫🇷 {phrase}")
        print(f"🇬🇧 {result['translation']}")
        print(f"   ({result['latency_ms']} ms)\n")
        latencies.append(result['latency_ms'])
    else:
        print(f"❌ Failed: {phrase} - {result['error']}\n")

if latencies:
    avg_latency = sum(latencies) / len(latencies)
    print(f"\n📊 Average Latency: {avg_latency:.2f} ms")
    print(f"📊 Min Latency: {min(latencies):.2f} ms")
    print(f"📊 Max Latency: {max(latencies):.2f} ms")

## Step 5: Edge Cases - Numbers and Dates

SentinelTranslate includes **number consistency** checks to ensure numbers aren't hallucinated.

Let's test sentences with numbers and dates.

In [ ]:
number_tests = [
    "J'ai 25 ans.",
    "La réunion est à 14h30.",
    "Le prix est de 1 299,99 euros.",
    "Nous avons vendu 10 000 unités en 2023.",
    "Le rapport compte 3 tableaux et 15 graphiques.",
    "Mon numéro de téléphone est 01 23 45 67 89."
]

print("Edge Cases: Numbers and Dates\n" + "="*70 + "\n")

for test in number_tests:
    result = translate_text(test)
    if result["success"]:
        print(f"🇫🇷 {test}")
        print(f"🇬🇧 {result['translation']}")
        print(f"   ✅ Number consistency check passed\n")
    else:
        print(f"🇫🇷 {test}")
        print(f"⚠️  {result['error']}\n")

## Step 6: Edge Cases - Named Entities

SentinelTranslate uses **NER (Named Entity Recognition)** to detect hallucinated entities.

Let's test sentences with people, places, and organizations.

In [ ]:
ner_tests = [
    "Marie travaille chez Google à Paris.",
    "Jean-Luc Picard est le capitaine de l'Enterprise.",
    "L'entreprise Apple a été fondée par Steve Jobs.",
    "Emmanuel Macron est le président de la France.",
    "La Tour Eiffel est située à Paris.",
    "Microsoft et Amazon sont des géants de la technologie."
]

print("Edge Cases: Named Entities (NER)\n" + "="*70 + "\n")

for test in ner_tests:
    result = translate_text(test)
    if result["success"]:
        print(f"🇫🇷 {test}")
        print(f"🇬🇧 {result['translation']}")
        print(f"   ✅ NER consistency check passed\n")
    else:
        print(f"🇫🇷 {test}")
        print(f"⚠️  {result['error']}\n")

## Step 7: Edge Cases - Special Characters

French uses accented characters (é, è, ê, à, ô, etc.). Let's test handling.

In [ ]:
accent_tests = [
    "C'est très intéressant!",
    "L'été est ma saison préférée.",
    "Voilà ce que j'ai trouvé.",
    "Où est la bibliothèque?",
    "Il a étudié à l'université de Montréal."
]

print("Edge Cases: Accented Characters\n" + "="*70 + "\n")

for test in accent_tests:
    result = translate_text(test)
    if result["success"]:
        print(f"🇫🇷 {test}")
        print(f"🇬🇧 {result['translation']}\n")
    else:
        print(f"🇫🇷 {test}")
        print(f"⚠️  {result['error']}\n")

## Step 8: Hallucination Detection Examples

SentinelTranslate has multi-layer safety checks:

1. **Length ratio check**: Translation shouldn't be >2.5x longer
2. **Repetition detection**: Repeated tokens indicate hallucination
3. **Language ID validation**: Ensure input matches declared language
4. **Number consistency**: All numbers must match
5. **NER consistency**: No new entities

Let's test some edge cases that might trigger safety checks.

In [ ]:
# Very short text (might have high length ratio)
print("Hallucination Test 1: Very Short Text\n" + "="*50)
result = translate_text("Oui.")
print_translation(result)

print("\n" + "="*70 + "\n")

# Very long sentence (stress test)
print("Hallucination Test 2: Long Sentence\n" + "="*50)
long_text = (
    "La transformation numérique des entreprises nécessite une approche stratégique "
    "qui combine l'innovation technologique, la formation des employés, et "
    "l'adaptation des processus métier pour rester compétitif dans un marché "
    "en constante évolution."
)
result = translate_text(long_text)
print_translation(result)

## Step 9: Performance Benchmarking

Let's measure translation performance across multiple requests.

In [ ]:
import statistics

# Benchmark with 20 translations
benchmark_phrases = [
    "Bonjour",
    "Comment ça va?",
    "Je suis développeur.",
    "Quel temps fait-il?",
    "Merci beaucoup."
]

print("Performance Benchmark\n" + "="*70 + "\n")
print("Running 20 translations (4 iterations of 5 phrases)...\n")

all_latencies = []
start_time = time.time()

for iteration in range(4):
    for phrase in benchmark_phrases:
        result = translate_text(phrase)
        if result["success"]:
            all_latencies.append(result['latency_ms'])

total_time = time.time() - start_time

if all_latencies:
    print(f"✅ Completed {len(all_latencies)} translations in {total_time:.2f} seconds\n")
    print(f"📊 Average Latency: {statistics.mean(all_latencies):.2f} ms")
    print(f"📊 Median Latency:  {statistics.median(all_latencies):.2f} ms")
    print(f"📊 Min Latency:     {min(all_latencies):.2f} ms")
    print(f"📊 Max Latency:     {max(all_latencies):.2f} ms")
    print(f"📊 Std Deviation:   {statistics.stdev(all_latencies):.2f} ms")
    print(f"\n📊 Throughput:      {len(all_latencies) / total_time:.2f} translations/second")
else:
    print("❌ Benchmark failed")

## Step 10: Error Handling

Let's test error scenarios and see how the API responds.

In [ ]:
print("Error Handling Tests\n" + "="*70 + "\n")

# Test 1: Empty text
print("Test 1: Empty Text")
result = translate_text("")
print(f"Result: {result.get('error', result.get('translation', 'Success'))}\n")

# Test 2: Wrong language (English instead of French)
print("Test 2: Wrong Language (should fail language ID check)")
result = translate_text("Hello, how are you?", source_lang="fr")  # English text with French label
if result["success"]:
    print(f"⚠️  Translation succeeded (language check may have passed): {result['translation']}\n")
else:
    print(f"✅ Language validation failed as expected: {result['error']}\n")

# Test 3: Very large text (might timeout)
print("Test 3: Very Large Text")
large_text = "Ceci est une phrase. " * 100  # 100 repetitions
result = translate_text(large_text, max_wait=10)
if result["success"]:
    print(f"✅ Large text translated successfully ({result['latency_ms']} ms)\n")
else:
    print(f"⚠️  {result['error']}\n")

## Summary

In this notebook, you learned:

✅ **Setup**: How to verify SentinelTranslate services are running  
✅ **Basic Translation**: Translating French text to English  
✅ **Common Phrases**: Greetings, business, and technical vocabulary  
✅ **Edge Cases**: Handling numbers, dates, named entities, special characters  
✅ **Safety Checks**: Understanding hallucination detection (length, repetition, NER, numbers)  
✅ **Performance**: Measuring latency and throughput  
✅ **Error Handling**: Dealing with empty text, wrong language, timeouts  

## Next Steps

1. **Try other languages**: Explore notebooks for Spanish, German, Russian, Chinese, etc.
2. **Batch translation**: See `batch_translation/batch_s3_example.ipynb` for large-scale workflows
3. **Add new models**: Use `model_conversion/convert_opus_to_onnx.ipynb` to deploy more language pairs
4. **Optimize performance**: Experiment with worker scaling and model quantization

## About OPUS-MT

The `opus-mt-fr-en` model is part of the [Helsinki-NLP OPUS-MT project](https://github.com/Helsinki-NLP/Opus-MT), which provides open-source neural machine translation models for 1000+ language pairs.

**Model details**:
- **Architecture**: Transformer (Marian NMT)
- **Training data**: OPUS corpus (parallel texts from web crawls, subtitles, etc.)
- **License**: CC-BY-4.0 (free for commercial use)
- **Performance**: BLEU score ~40 on WMT test sets

---

**Questions or issues?** Check the main [README](../README.md) or open an issue on GitHub.